In [1]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import psycopg2 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Load in password
from secret import secret, database, username, host

# Establish connection to database
conn = psycopg2.connect(
    host=host,
    database=database,
    user= username,
    password= secret
)

In [ ]:
# Load the data
query = 'SELECT * FROM clark_co_traffic'
traffic_df = pd.read_sql_query(query, conn)

In [ ]:
# Show data
traffic_df.head()

In [3]:
# # Separate data into features and target
# # NEED TO CLARIFY TARGET(S)

# y = df['TARGET NAME']
# X = dr.drop(columns = 'TARGET NAME', 1)

In [4]:
# # Split dataset into training and testing

# # Do we need a random_state here? Maybe?
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify = y)
# X_train.shape

In [6]:
# # Train the model
# model.fit(X_train, y_train)

In [7]:
# # Create predictions
# y_pred = model.predict(X_test)
# results = pd.DataFrame({
#     "Prediction": y_pred,
#     "Actual": y_test
# }).reset_index(drop = True)
# results.head()

In [8]:
# # Assess accuracy score
# accuracy_score(y_test, y_pred)

In [9]:
# # Generate confusion matrix
# confusion_matrix(y_test, y_pred)

In [ ]:
# # Print classification report
# print(classification_report(y_test, y_pred)